In [2]:
    # %reload_ext XXX.mumble
    import CommonMark
    from IPython import get_ipython
    from IPython.core.inputsplitter import IPythonInputSplitter
    from IPython.core.compilerop import CachingCompiler
    from dataclasses import dataclass, field
    from textwrap import indent, dedent

In [3]:
    def identity(object=None, *_, **__): return object

In [4]:
    def codify(ast, *, eval = """""", parser=__import__('CommonMark').Parser(), transform=dedent) -> str:
        """Convert markdown to code with the correct line numbers and positions in a string.

        This function replaces non-code_block nodes with blanklines.

        >>> code = codify('''# Markdown\\n\\tprint(10)''')
        
        The {code} is valid
        >>> assert compile(code, '<test>', 'exec')
        """
        if isinstance(ast, str): ast = parser.parse(ast)
        for node, _ in ast.walker():
            if node.t == 'code_block':
                line, col = node.sourcepos[0]
                end = line + len(node.literal.splitlines())
                lines = node.literal.splitlines()
                while len(eval.splitlines()) < (line-1):
                    if not eval and eval is not 0:
                        eval += (' '*col)+'#'
                    eval += '\n'
                eval += indent(node.literal, ' '*col)
        return dedent(transform(eval))

In [5]:
    class Display(CommonMark.Parser):
        def parse(self, source):
            if source and source.splitlines()[0].strip():
                from IPython.display import display, Markdown
                display(Markdown(source))
            return super().parse(source)

In [6]:
        class Markdown(CachingCompiler): 
            input_transformer_manager = None
            def __init__(self, parser=None):
                super().__init__()
                self.parser = parser or Display()
                self.load()
                    
            def load(self):
                if self.input_transformer_manager is not None: self.unload()
                self.input_transformer_manager = get_ipython().input_transformer_manager
                get_ipython().input_transformer_manager = IPythonInputSplitter(
                    logical_line_transforms=[], physical_line_transforms=[], python_line_transforms=[])
                return self

            def unload(self): 
                get_ipython().input_transformer_manager = self.input_transformer_manager
                self.input_transformer_manager = None

            def __post_init__(Markdown): CachingCompiler.__init__(Markdown)

            def ast_parse(self, data, filename='<unknown>', symbol='exec'): 
                code = codify(data, parser=self.parser, transform=self.transform)
                return super().ast_parse(code, filename=filename, symbol=symbol)

            @property
            def transform(self): return (
                self.input_transformer_manager and self.input_transformer_manager.transform_cell or identity)

In [7]:
    def load_ipython_extension(ip=get_ipython()): 
        ip.compile = Markdown().load()

        
    def unload_ipython_extension(ip=get_ipython()): 
        ip.compile = getattr(ip.compile, 'unload', identity)() or CachingCompiler()
        

In [8]:
    if __name__ == '__main__': 
        __import__('doctest').testmod()
        load_ipython_extension()